In [ ]:
import csv
import heapq
import networkx as nx
import matplotlib.pyplot as plt
import zipfile
import os
from collections import defaultdict, namedtuple

import logging

# Create a logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Create a handler to display logs in Jupyter notebook's output cell
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

# Format the logs
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

## 1. Extract gtfs from zip file

In [ ]:

def extract_gtfs_zip(zip_path, extract_to='gtfs_feed'):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Extract the GTFS feed
zip_path = '../data/study_area_gtfs_bus.zip'
extract_to = 'gtfs_feed'
extract_gtfs_zip(zip_path, extract_to)

## 2. Parse gtfs feed

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

Edge = namedtuple('Edge', ['to', 'weight', 'start_time', 'end_time', 'route'])

def parse_gtfs(extract_to, crs='EPSG:3857'):
    stops = {}
    routes = {}
    trips = {}
    stop_times = defaultdict(list)

    stops_df = pd.read_csv(os.path.join(extract_to, "stops.txt"))
    stops_gdf = gpd.GeoDataFrame(stops_df, geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat))
    stops_gdf.set_crs('EPSG:4326', inplace=True)
    stops_gdf = stops_gdf.to_crs(crs)
    for index, row in stops_gdf.iterrows():
        row_dict = row.to_dict()
        row_dict['stop_lon'], row_dict['stop_lat'] = row.geometry.x, row.geometry.y
        stops[row['stop_id']] = row_dict

    with open(os.path.join(extract_to, "routes.txt"), mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            routes[row['route_id']] = row

    with open(os.path.join(extract_to, "trips.txt"), mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            trips[row['trip_id']] = row

    with open(os.path.join(extract_to, "stop_times.txt"), mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            stop_times[row['trip_id']].append(row)

    return stops, routes, trips, stop_times

In [ ]:
# Apply
stops, routes, trips, stop_times = parse_gtfs(extract_to, crs='EPSG:3857')

In [ ]:
stop_times

## 3. Generate transfer graph

In [ ]:
stops

In [ ]:
list(stops.values())

In [ ]:
from sklearn.neighbors import KDTree

stops_list = list(stops.values())

stops_location = []
for stop in stops_list:
    if isinstance(stop, dict) and 'stop_lat' in stop and 'stop_lon' in stop:
        stops_location.append((float(stop['stop_lat']), float(stop['stop_lon'])))

stops_location

if not stops_location:
    raise ValueError("No valid stops found")


In [ ]:
btree = KDTree(stops_location)

# check values in KDTRree
btree.query([stops_location[0]], k=20)

In [ ]:
indices, distances = btree.query_radius(stops_location, r= 300, return_distance=True)


In [ ]:
indices

In [ ]:
distances

In [ ]:
not stops_location

In [ ]:
def generate_transfers(stops, max_distance_meters, walk_speed_kmph):
    if walk_speed_kmph == 0:
        raise ValueError("walk_speed_kmph cannot be zero")

    transfers = defaultdict(list)
    stops_list = list(stops.values())
    stops_location = []
    for stop in stops_list:
        if isinstance(stop, dict) and 'stop_lat' in stop and 'stop_lon' in stop:
            stops_location.append((float(stop['stop_lat']), float(stop['stop_lon'])))


    if not stops_location:
        raise ValueError("No valid stops found")


    btree = KDTree(stops_location)

    crow_flies_distance_factor = 1.5
    indices, distances = btree.query_radius(stops_location, r=max_distance_meters, return_distance=True)

    for from_idx, (to_indices, ds) in enumerate(zip(indices, distances)):
        for to_idx, d in zip(to_indices, ds):
            if from_idx != to_idx:
                walk_dis = d * crow_flies_distance_factor
                walk_h = walk_dis / 1000 / walk_speed_kmph
                walk_sec = walk_h * 60 * 60

                transfers[stops_list[from_idx]['stop_id']].append({
                    'from_stop_id': stops_list[from_idx]['stop_id'],
                    'to_stop_id': stops_list[to_idx]['stop_id'],
                    'min_transfer_time': walk_sec
                })

    return transfers

In [ ]:
# Generate transfers
transfers = generate_transfers(stops, max_distance_meters= 1000, walk_speed_kmph= 5)
transfers

## 4. Build the graph

In [ ]:
class Graph:
    def __init__(self):
        self.adj_list = defaultdict(list)

    def add_edge(self, from_node, to_node, weight, start_time, end_time, route):
        self.adj_list[from_node].append(Edge(to_node, weight, start_time, end_time, route))

    def get_nodes(self):
        return list(self.adj_list.keys())

    def get_edges(self):
        edges = []
        for from_node, edges_list in self.adj_list.items():
            for edge in edges_list:
                edges.append((from_node, edge.to, edge.weight, edge.start_time, edge.end_time, edge.route))
        return edges

def build_graph_from_gtfs(stops, routes, trips, stop_times, transfers):
    graph = Graph()
    
    # Add edges from stop_times
    for trip_id, times in stop_times.items():
        times.sort(key=lambda x: x['stop_sequence'])  # Ensure stop_times are sorted by stop_sequence
        for i in range(len(times) - 1):
            from_stop = times[i]['stop_id']
            to_stop = times[i + 1]['stop_id']
            h, m, s = map(int, times[i]['arrival_time'].split(':'))
            start_time = h * 3600 + m * 60 + s
            h, m, s = map(int, times[i + 1]['departure_time'].split(':'))
            end_time = h * 3600 + m * 60 + s
            weight = end_time - start_time
            route_id = trips[trip_id]['route_id']
            graph.add_edge(from_stop, to_stop, weight, start_time, end_time, route_id)
    
    # Add transfer edges
    for from_stop, trans in transfers.items():
        for transfer in trans:
            to_stop = transfer['to_stop_id']
            min_transfer_time = int(transfer['min_transfer_time'])
            graph.add_edge(from_stop, to_stop, weight=min_transfer_time, start_time=0, end_time=float('inf'), route='transfer')

    return graph


In [ ]:
# Build the graph from GTFS data
graph = build_graph_from_gtfs(stops, routes, trips, stop_times, transfers)


In [ ]:
nodes = graph.get_nodes()
for node in nodes:
    print(f"Node: {node}")


In [ ]:
edges = graph.get_edges()
for edge in edges:
    print(f"Edge: {edge}")

## 5. Modified djikstra algorithm

In [ ]:
def dijkstra_with_transfers(graph, start, start_time, max_transfers=3):

    # Check if the start node is in the graph
    if start not in graph.adj_list:
        print("Invalid start node. Valid nodes are:")
        for node in graph.adj_list:
            print(node)
        return None
    # Initialize distances, routes, and priority queue
    distances = {node: float('inf') for node in graph.adj_list}
    distances[start] = 0
    priority_queue = [(start_time, start, set(), 0)]  # (time, node, routes, transfers)
    shortest_path_tree = {}
    routes_used = {node: set() for node in graph.adj_list}
    transfers = {node: float('inf') for node in graph.adj_list}
    transfers[start] = 0

    while priority_queue:
        current_time, current_node, current_routes, current_transfers = heapq.heappop(priority_queue)

        # If the popped node has a time greater than the currently known shortest time, skip it
        if current_time > distances[current_node]:
            continue

        # Examine and relax edges with time windows and route tracking
        for edge in graph.adj_list[current_node]:
            if current_time <= edge.end_time:
                # Calculate the wait time if necessary
                wait_time = max(0, edge.start_time - current_time)
                arrival_time = current_time + wait_time + edge.weight

                # Calculate the number of transfers
                new_transfers = current_transfers
                if edge.route not in current_routes:
                    new_transfers += 1

                # Check if the number of transfers exceeds the maximum allowed
                if new_transfers > max_transfers:
                    continue

                # If this path is shorter or uses fewer transfers, update the time, routes, and transfers
                if arrival_time < distances[edge.to] or (arrival_time == distances[edge.to] and new_transfers < transfers[edge.to]):
                    distances[edge.to] = arrival_time
                    routes_used[edge.to] = current_routes | {edge.route}
                    transfers[edge.to] = new_transfers
                    heapq.heappush(priority_queue, (arrival_time, edge.to, current_routes | {edge.route}, new_transfers))
                    shortest_path_tree[edge.to] = (current_node, edge.route)

    return distances, shortest_path_tree, routes_used, transfers

def print_shortest_path_tree(shortest_path_tree, routes_used, start):
    print("Shortest Path Tree:")
    for node, (parent, _) in shortest_path_tree.items():
        path = [node]
        while parent != start:
            path.append(parent)
            parent, _ = shortest_path_tree[parent]
        path.append(start)
        path.reverse()
        print(f"Destination Node: {node}, Path: {' -> '.join(path)}, Routes: {', '.join(routes_used[node])}")


# Create a graph



In [ ]:
def dijkstra_with_transfers(graph, start, start_time, max_transfers=3):
    logging.info(f"Running Dijkstra's algorithm with start node {start}, start time {start_time}, and max transfers {max_transfers}")

    if start not in graph.adj_list:
        logging.error("Invalid start node. Valid nodes are:")
        for node in graph.adj_list:
            logging.error(node)
        return None

    distances = {node: float('inf') for node in graph.adj_list}
    distances[start] = start_time
    priority_queue = [(start_time, start, set(), 0)]
    shortest_path_tree = {}
    routes_used = {node: set() for node in graph.adj_list}
    transfers = {node: float('inf') for node in graph.adj_list}
    transfers[start] = 0

    logging.debug(f"Initial distances: {distances}")
    logging.debug(f"Initial priority queue: {priority_queue}")
    logging.debug(f"Initial routes used: {routes_used}")
    logging.debug(f"Initial transfers: {transfers}")

    while priority_queue:
        logger.debug(f"Priority queue: {priority_queue}")
        current_time, current_node, current_routes, current_transfers = heapq.heappop(priority_queue)
        logging.debug(f"Popped node {current_node} with time {current_time}, routes {current_routes}, and transfers {current_transfers}")

        if current_time > distances[current_node]:
            logging.debug(f"Skipping node {current_node} because its time {current_time} is greater than the known shortest time {distances[current_node]}")
            continue

        for edge in graph.adj_list[current_node]:
            if current_time <= edge.end_time:
                wait_time = max(0, edge.start_time - current_time)
                arrival_time = current_time + wait_time + edge.weight
                logging.debug(f"Examining edge to node {edge.to} with start time {edge.start_time}, end time {edge.end_time}, and weight {edge.weight}")
                logging.debug(f"Calculated wait time: {wait_time}, arrival time: {arrival_time}")

                new_transfers = current_transfers
                if edge.route not in current_routes:
                    new_transfers += 1
                    logging.debug(f"New route {edge.route} requires a transfer, increasing transfers to {new_transfers}")

                if new_transfers > max_transfers:
                    logging.debug(f"Skipping edge to node {edge.to} because it requires {new_transfers} transfers, which exceeds the maximum of {max_transfers}")
                    continue

                if arrival_time < distances[edge.to] or (arrival_time == distances[edge.to] and new_transfers < transfers[edge.to]):
                    distances[edge.to] = arrival_time
                    routes_used[edge.to] = current_routes | {edge.route}
                    transfers[edge.to] = new_transfers
                    heapq.heappush(priority_queue, (arrival_time, edge.to, current_routes | {edge.route}, new_transfers))
                    shortest_path_tree[edge.to] = (current_node, edge.route)
                    logging.debug(f"Updated shortest path to node {edge.to} with time {arrival_time}, routes {routes_used[edge.to]}, and transfers {transfers[edge.to]}")

    logging.debug(f"Final distances: {distances}")
    logging.debug(f"Final shortest path tree: {shortest_path_tree}")
    logging.debug(f"Final routes used: {routes_used}")
    logging.debug(f"Final transfers: {transfers}")

    return distances, shortest_path_tree, routes_used, transfers

In [ ]:
# get node ids from the graph
graph

In [23]:
# Apply Dijkstra's algorithm
start_node = '450011574'
start_time = (11 * 3600) + (30* 60)  # seconds from midnight
# Apply Dijkstra's algorithm with transfers
distances = dijkstra_with_transfers(graph, start_node, start_time, max_transfers= 1)

# Display or use results as needed
print("Shortest distances from start node:", distances)


KeyboardInterrupt: 

In [ ]:
dijkstra_with_transfers(graph, start_node, start_time, max_transfers= 3)

In [ ]:
# check if any of the values in "distances" are not infinity
distances_dict = distances[0]
unique_values = set(distances_dict.values())
unique_values



In [ ]:
def visualize_graph(distances, shortest_path_tree, routes_used, start_node):
    # Create a new NetworkX graph
    G = nx.Graph()

    # Add nodes to the graph
    for node, distance in distances.items():
        G.add_node(node, distance=distance)

    # Add edges to the graph
    for node, (parent, route) in shortest_path_tree.items():
        if parent is not None:
            G.add_edge(parent, node, route=route)

    # Assign colors to nodes based on travel time
    node_colors = [distances[node] for node in G.nodes()]

    # Draw the graph
    pos = nx.spring_layout(G)  # or use other layout algorithms
    nx.draw(G, pos, with_labels=True, node_color=node_colors, cmap=plt.cm.plasma)

    # Add color bar
    sm = plt.cm.ScalarMappable(cmap=plt.cm.plasma, norm=plt.Normalize(vmin=min(node_colors), vmax=max(node_colors)))
    sm._A = []  # fake up the array of the scalar mappable
    plt.colorbar(sm)

    # Show plot
    plt.title('Network Visualization with Travel Time')
    plt.show()


In [ ]:
# Example usage with distances, shortest_path_tree, routes_used from dijkstra_with_routes
visualize_graph(distances, shortest_path_tree, routes_used, start_node = '3200YND56740')